In [ ]:
"""
%pip install transformers peft datasets accelerate bitsandbytes
%pip install -U trl
%pip install -U transformers peft trl accelerate
"""

In [ ]:
import gc
gc.collect()
import torch
torch.cuda.empty_cache()

In [ ]:
import os
import torch
from huggingface_hub import login, whoami 

# Set the environment variable for the Hugging Face cache directory
os.environ["HF_HUB_CACHE"] = "/data/llm_cyber/hf_cache_2"
custom_cache_dir = "/data/llm_cyber/hf_cache_2"

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub.constants import HF_HUB_CACHE # Import HF_HUB_CACHE last

# Log in to the Hugging Face Hub using a personal access token
login("HF_TOKEN")

In [ ]:
import os
import torch
from huggingface_hub import login, whoami 

# Set the environment variable for the Hugging Face cache directory
os.environ["HF_HUB_CACHE"] = "/data/llm_cyber/hf_cache_2"
custom_cache_dir = "/data/llm_cyber/hf_cache_2"

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub.constants import HF_HUB_CACHE # Import HF_HUB_CACHE last

# Log in to the Hugging Face Hub using a personal access token
login("HF_TOKEN")

custom_cache_dir = "/data/llm_cyber/hf_cache_2"

# 4-bit quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# LOAD THE MODEL: USE THE cache_dir ARGUMENT
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    cache_dir=custom_cache_dir
)

# Load the tokenizer: Also use the cache_dir argument
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=custom_cache_dir)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
from peft import LoraConfig

# Standard LoRA Configuration
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# Load your dataset
# Load a dataset from a local JSON Lines file, specifying the 'train' split
dataset = load_dataset("json", data_files="/data/llm_cyber/normalized_merged_data.jsonl", split="train")

# Function to format data according to the Mistral structure
def format_data_for_mistral(example):
    # Format the data into the ChatML-like structure required by Mistral models
    text = f"<s>[INST] {example['instruction']} [/INST] {example['response']}</s>"
    return {"text": text}

# Apply the formatting function
formatted_dataset = dataset.map(format_data_for_mistral)

# Configure the training arguments
training_args = TrainingArguments(
    output_dir="./mistral-mixlora-finetune",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-2,
    # num_train_epochs=1,
    max_steps=1000,
    logging_steps=1,
    report_to="wandb",
    gradient_checkpointing=True,
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    peft_config=peft_config,
    args=training_args,
)

# Start training
trainer.train()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import os

# --- CẤU HÌNH CỤ THỂ CỦA BẠN ---
# 1. Tên/ID của model nền Mistral (base model)
BASE_MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"

# 2. Đường dẫn tới checkpoint LoRA của bạn
LORA_ADAPTER_PATH = "/data/llm_cyber/mistral-mixlora-finetune-3/checkpoint-300"

# 3. Đường dẫn lưu model đã hợp nhất (CHỈ ĐỊNH TRỰC TIẾP VÀO /data)
# Model sẽ được lưu vào: /data/mistral_merged_model_gguf_ready
OUTPUT_MERGED_MODEL_DIR = "/data/llm_cyber/mistral_merged_model_gguf_ready"

# Đảm bảo thư mục đầu ra tồn tại
os.makedirs(OUTPUT_MERGED_MODEL_DIR, exist_ok=True)
# ---------------------------------


In [ ]:
import torch
# Dọn dẹp bộ nhớ GPU trước khi tải model
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("Đã dọn dẹp bộ nhớ CUDA.")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import os

# --- CẤU HÌNH CỤ THỂ CỦA BẠN ---
BASE_MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"
LORA_ADAPTER_PATH = "/data/llm_cyber/mistral-mixlora-finetune-3/checkpoint-300"
OUTPUT_MERGED_MODEL_DIR = "/data/mistral_merged_model_gguf_ready"
HF_CACHE_DIR = "/data/llm_cyber/hf_cache_2" # Thư mục cache

# Đảm bảo thư mục đầu ra tồn tại
os.makedirs(OUTPUT_MERGED_MODEL_DIR, exist_ok=True)

# Dọn dẹp VRAM (cần thiết nếu muốn chạy các bước sau)
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("Đã dọn dẹp bộ nhớ CUDA.")
# ---------------------------------

# --- BƯỚC 1: Tải Model Gốc và Adapter ---
print(f"1. Đang tải model nền: {BASE_MODEL_ID} VÀO CPU/RAM (để tránh lỗi VRAM)...")

# Sử dụng cache_dir để đảm bảo model được tải vào /data
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, cache_dir=HF_CACHE_DIR)

# FIX QUAN TRỌNG: Tải HOÀN TOÀN model vào CPU RAM
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    torch_dtype=torch.float16, # Tải ở FP16
    device_map="cpu",         # TẢI HOÀN TOÀN VÀO CPU RAM
    low_cpu_mem_usage=True,
    cache_dir=HF_CACHE_DIR
)

# Tải adapter LoRA từ checkpoint đã cho
print(f"2. Đang tải adapter LoRA từ: {LORA_ADAPTER_PATH}...")
model = PeftModel.from_pretrained(base_model, LORA_ADAPTER_PATH)

# --- BƯỚC 2: Hợp nhất (Mix/Merge) Adapter ---
print("3. Đang hợp nhất adapter LoRA vào model nền (trên CPU/RAM)...")
# Quá trình này hoàn toàn diễn ra trên CPU/RAM và có thể mất vài phút.
model = model.merge_and_unload()

# --- BƯỚC 3: Lưu Model đã Hợp nhất ---
print(f"4. Đang lưu model đã hợp nhất (dạng FP16) tại: {OUTPUT_MERGED_MODEL_DIR}...")
# Model này sẽ nặng khoảng 15GB và được lưu vào /data
model.save_pretrained(OUTPUT_MERGED_MODEL_DIR, safe_serialization=True)
tokenizer.save_pretrained(OUTPUT_MERGED_MODEL_DIR)

print("✅ Hoàn tất hợp nhất! Model đã sẵn sàng để chuyển đổi GGUF.")
print(f"Model đã được lưu tại: {OUTPUT_MERGED_MODEL_DIR}")